In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import pearsonr
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV
import seaborn as sns

In [5]:
df_total = pd.read_csv(r'C:\Users\VT.33\Desktop\COVID19_VN_MOH_MLmodels\Predicting LOS (Length of Stay)\assets\Processed data\df_total.csv', encoding="utf-8")

df_train = pd.read_csv(r'C:\Users\VT.33\Desktop\COVID19_VN_MOH_MLmodels\Predicting LOS (Length of Stay)\assets\Processed data\df_train.csv', encoding="utf-8")

df_test = pd.read_csv(r'C:\Users\VT.33\Desktop\COVID19_VN_MOH_MLmodels\Predicting LOS (Length of Stay)\assets\Processed data\df_test.csv', encoding="utf-8")

In [13]:
df_total.info()
df_total.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AGE                 163 non-null    float64
 1   SEX                 163 non-null    int64  
 2   ORIGIN_EUROPE       163 non-null    int64  
 3   ORIGIN_CHINA        163 non-null    int64  
 4   ORIGIN_US           163 non-null    int64  
 5   ORIGIN_DOMESTIC     163 non-null    int64  
 6   DAYS_OF_ONSET       163 non-null    int64  
 7   ONSET_FEVER         163 non-null    int64  
 8   ONSET_COUGH         163 non-null    int64  
 9   ONSET_TIRED         163 non-null    int64  
 10  ONSET_THROAT        163 non-null    int64  
 11  ONSET_RES           163 non-null    int64  
 12  BACKGROUND_DISEASE  163 non-null    int64  
 13  HOSPITAL            163 non-null    int64  
 14  LOS                 16 non-null     float64
dtypes: float64(2), int64(13)
memory usage: 19.2 KB


,AGE,SEX,ORIGIN_EUROPE,ORIGIN_CHINA,ORIGIN_US,ORIGIN_DOMESTIC,DAYS_OF_ONSET,ONSET_FEVER,ONSET_COUGH,ONSET_TIRED,ONSET_THROAT,ONSET_RES,BACKGROUND_DISEASE,HOSPITAL,LOS
0,66.0,1,0,1,0,0,6,0,0,0,0,0,2,1,20.0
1,28.0,1,0,1,0,0,3,0,0,0,0,0,0,1,12.0
2,25.0,0,0,1,0,0,7,0,0,0,0,0,0,0,4.0
3,29.0,1,0,1,0,0,9,0,0,0,0,0,0,0,19.0
4,23.0,0,0,1,0,0,5,0,0,0,0,0,0,0,11.0


In [12]:
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AGE                 147 non-null    int64  
 1   SEX                 147 non-null    int64  
 2   ORIGIN_EUROPE       147 non-null    int64  
 3   ORIGIN_CHINA        147 non-null    int64  
 4   ORIGIN_US           147 non-null    int64  
 5   ORIGIN_DOMESTIC     147 non-null    int64  
 6   DAYS_OF_ONSET       147 non-null    int64  
 7   ONSET_FEVER         147 non-null    int64  
 8   ONSET_COUGH         147 non-null    int64  
 9   ONSET_TIRED         147 non-null    int64  
 10  ONSET_THROAT        147 non-null    int64  
 11  ONSET_RES           147 non-null    int64  
 12  BACKGROUND_DISEASE  147 non-null    int64  
 13  HOSPITAL            147 non-null    int64  
 14  LOS                 0 non-null      float64
dtypes: float64(1), int64(14)
memory usage: 17.4 KB


,AGE,SEX,ORIGIN_EUROPE,ORIGIN_CHINA,ORIGIN_US,ORIGIN_DOMESTIC,DAYS_OF_ONSET,ONSET_FEVER,ONSET_COUGH,ONSET_TIRED,ONSET_THROAT,ONSET_RES,BACKGROUND_DISEASE,HOSPITAL,LOS
0,26,0,1,0,0,0,5,1,1,1,0,0,0,0,NaN
1,27,1,0,0,0,0,7,0,1,0,1,0,0,0,NaN
2,27,0,0,0,0,1,0,0,0,0,0,0,0,0,NaN
3,64,1,0,0,0,1,0,0,0,0,0,0,1,0,NaN
4,61,1,1,0,0,0,2,0,1,1,0,0,0,0,NaN


In [11]:
df_test.info()
df_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AGE                 16 non-null     float64
 1   SEX                 16 non-null     int64  
 2   ORIGIN_EUROPE       16 non-null     int64  
 3   ORIGIN_CHINA        16 non-null     int64  
 4   ORIGIN_US           16 non-null     int64  
 5   ORIGIN_DOMESTIC     16 non-null     int64  
 6   DAYS_OF_ONSET       16 non-null     int64  
 7   ONSET_FEVER         16 non-null     int64  
 8   ONSET_COUGH         16 non-null     int64  
 9   ONSET_TIRED         16 non-null     int64  
 10  ONSET_THROAT        16 non-null     int64  
 11  ONSET_RES           16 non-null     int64  
 12  BACKGROUND_DISEASE  16 non-null     int64  
 13  HOSPITAL            16 non-null     int64  
 14  LOS                 16 non-null     int64  
dtypes: float64(1), int64(14)
memory usage: 2.0 KB


,AGE,SEX,ORIGIN_EUROPE,ORIGIN_CHINA,ORIGIN_US,ORIGIN_DOMESTIC,DAYS_OF_ONSET,ONSET_FEVER,ONSET_COUGH,ONSET_TIRED,ONSET_THROAT,ONSET_RES,BACKGROUND_DISEASE,HOSPITAL,LOS
0,66.0,1,0,1,0,0,6,0,0,0,0,0,2,1,20
1,28.0,1,0,1,0,0,3,0,0,0,0,0,0,1,12
2,25.0,0,0,1,0,0,7,0,0,0,0,0,0,0,4
3,29.0,1,0,1,0,0,9,0,0,0,0,0,0,0,19
4,23.0,0,0,1,0,0,5,0,0,0,0,0,0,0,11


In [14]:
LOS = df_train['LOS'].values
LOS

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan])